In [1]:

import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf
from pennylane.templates import RandomLayers
#(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
#(x_train, y_train), (x_test, y_test)=tf.keras.datasets.cifar10.load_data()
n_epochs = 30
n_layers = 1
n_train =50
n_test =30
n_qubits=4

PREPROCESS = True
np.random.seed(0)
tf.random.set_seed(0)

train_images = x_train[:n_train]
train_labels = y_train[:n_train]
test_images = x_test[:n_test]
test_labels = y_test[:n_test]

train_images = train_images / 255
test_images = test_images / 255

train_images = np.array(train_images[..., tf.newaxis],requires_grad=False)
test_images = np.array(test_images[..., tf.newaxis],requires_grad=False)

dev = qml.device("qiskit.ibmq", wires=4, backend='ibmq_santiago',ibmqx_token="a0b27f4ef8b70f52ef65107ca5f4dffdb828ef1dd74a87dcf7d5c2894da111800698348c2e5ab6ff372517d7dd14d7581cb182729f18330dae09500eeab9af82")
rand_params = np.random.uniform(high=2 * np.pi, size=(n_layers, 4))
params = 2 * np.pi * tf.random.uniform([2,8])
@qml.qnode(dev)
def circuit(data):
    for i in range(0, n_qubits):
        qml.RX(np.pi * data[i], wires=i)
    
    for i in range(0, n_qubits):
        qml.RX(params[0,i], wires=i)
        qml.RZ(params[0,i+4], wires=i)

    for i in range(0, n_qubits - 1, 2):
       
        qml.CRX(params[0,i], wires=[i, i + 1])
    for i in range(0, n_qubits):
        qml.RX(params[1,i], wires=i)
        qml.RZ(params[1,i+4], wires=i)

    for i in range(1, n_qubits - 1, 2):
       
        qml.CRX(params[1,i], wires=[i, i + 1])

    return [qml.expval(qml.PauliZ(w)) for w in range(4)]



SAVE_PATH=r"C:\Users\tejal\Documents\Quatum_op\oom"
def quanv(image):
    """Convolves the input image with many applications of the same quantum circuit."""
    out = np.zeros((14, 14, 4))
    
    for j in range(0, 28, 2):
        for k in range(0, 28, 2):
            
            
             q_results = circuit(
                [ image[j, k, 0],
                  image[j, k + 1, 0],
                  image[j + 1, k, 0],
                  image[j + 1, k + 1, 0]
                ]

             )
             for i in range(4):
                  out[j // 2, k // 2, i] = q_results[i]

      

        return out

if PREPROCESS == True:
    q_train_images = []
    print("Quantum pre-processing of train images:")
    for idx, img in enumerate(train_images):
        print("{}/{}        ".format(idx + 1, n_train), end="\r")


        q_train_images.append(quanv(img))
    q_train_images = np.asarray(q_train_images)

    q_test_images = []
    print("\nQuantum pre-processing of test images:")
    for idx, img in enumerate(test_images):
        print("{}/{}        ".format(idx + 1, n_test), end="\r")
        q_test_images.append(quanv(img))
    q_test_images = np.asarray(q_test_images)

   




model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(10, activation="softmax")
    ])

opt=tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(
        optimizer=opt,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],)




from tensorflow import keras
import matplotlib.pyplot as plt
n_epochs=30


q_history = model.fit(q_train_images,train_labels,validation_data=(q_test_images, test_labels),batch_size=16,epochs=n_epochs,verbose=2)
score = model.evaluate(q_test_images, test_labels, verbose=0)


model1 = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(filters=4, kernel_size=(2,2),input_shape=(28,28,1),activation='relu'),
        tf.keras.layers.Conv2D(filters=4, kernel_size=(2,2),activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dropout(0.4),

        tf.keras.layers.Dense(10, activation="softmax")
    ])
opt=tf.keras.optimizers.Adam(learning_rate=0.01)
model1.compile(
        optimizer=opt,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],)
from tensorflow import keras
import matplotlib.pyplot as plt
n_epochs=30


c_history = model1.fit(
    train_images,
    train_labels,
    validation_data=(test_images, test_labels),
    batch_size=16,
    epochs=n_epochs,
    verbose=2,
)



Quantum pre-processing of train images:


C:\Users\tejal\Anaconda3\New folder\envs\tf\lib\site-packages\pennylane\math\multi_dispatch.py:65: UserWarning: Contains tensors of types {'autograd', 'tensorflow'}; dispatch will prioritize TensorFlow and PyTorch over autograd. Consider replacing Autograd with vanilla NumPy.
  warnings.warn(
C:\Users\tejal\Anaconda3\New folder\envs\tf\lib\site-packages\pennylane_qiskit\qiskit_device.py:428: UserWarning: ibmqx_token is not a recognized runtime option and may be ignored by the backend.
  self._current_job = self.backend.run(compiled_circuits, shots=self.shots, **self.run_args)
C:\Users\tejal\Anaconda3\New folder\envs\tf\lib\site-packages\pennylane\math\multi_dispatch.py:65: UserWarning: Contains tensors of types {'autograd', 'tensorflow'}; dispatch will prioritize TensorFlow and PyTorch over autograd. Consider replacing Autograd with vanilla NumPy.
  warnings.warn(


50/50        
Quantum pre-processing of test images:
Epoch 1/30   
4/4 - 0s - loss: 2.2883 - accuracy: 0.1600 - val_loss: 2.1930 - val_accuracy: 0.2333
Epoch 2/30
4/4 - 0s - loss: 2.1342 - accuracy: 0.2200 - val_loss: 2.1088 - val_accuracy: 0.2333
Epoch 3/30
4/4 - 0s - loss: 2.0289 - accuracy: 0.2400 - val_loss: 2.0777 - val_accuracy: 0.1667
Epoch 4/30
4/4 - 0s - loss: 1.9239 - accuracy: 0.3600 - val_loss: 1.9965 - val_accuracy: 0.2667
Epoch 5/30
4/4 - 0s - loss: 1.7631 - accuracy: 0.4200 - val_loss: 1.9031 - val_accuracy: 0.3000
Epoch 6/30
4/4 - 0s - loss: 1.8217 - accuracy: 0.3800 - val_loss: 1.8838 - val_accuracy: 0.2333
Epoch 7/30
4/4 - 0s - loss: 1.6562 - accuracy: 0.3600 - val_loss: 1.8280 - val_accuracy: 0.2667
Epoch 8/30
4/4 - 0s - loss: 1.5434 - accuracy: 0.4800 - val_loss: 1.7887 - val_accuracy: 0.3333
Epoch 9/30
4/4 - 0s - loss: 1.5207 - accuracy: 0.4400 - val_loss: 1.7807 - val_accuracy: 0.2667
Epoch 10/30
4/4 - 0s - loss: 1.4482 - accuracy: 0.4600 - val_loss: 1.8116 - val_